## 抓取 JPY 歷史匯率

In [2]:
import pandas as pd

In [77]:
o_data = pd.read_html('http://rate.bot.com.tw/xrt/quote/l6m/JPY')
print(type(o_data))
print(len(o_data))

<class 'list'>
1


In [78]:
jpy_his = o_data[0].iloc[:,0:6]
jpy_his.sample(5)

掛牌日期                幣別    現金匯率 Unnamed: 4_level_0  \
           本行買入      本行賣出    本行買入    本行賣出 Unnamed: 4_level_1   
120  2017/08/04  日圓 (JPY)  0.2658  0.2768             0.2722   
22   2017/12/22  日圓 (JPY)  0.2558  0.2668             0.2622   
40   2017/11/28  日圓 (JPY)  0.2611  0.2721             0.2675   
81   2017/09/28  日圓 (JPY)  0.2609  0.2719             0.2673   
29   2017/12/13  日圓 (JPY)  0.2563  0.2673             0.2627   

                  即期匯率  
    Unnamed: 5_level_1  
120             0.2762  
22              0.2662  
40              0.2715  
81              0.2713  
29              0.2667

In [79]:
jpy_his.columns = ['掛牌日期', '幣別', '現金-本行買入', '現金-本行賣出', '即期-本行買入', '即期-本行賣出']
jpy_his.sample(5)

,掛牌日期,幣別,現金-本行買入,現金-本行賣出,即期-本行買入,即期-本行賣出
25,2017/12/19,日圓 (JPY),0.2581,0.2691,0.2645,0.2685
41,2017/11/27,日圓 (JPY),0.2610,0.2720,0.2674,0.2714
115,2017/08/11,日圓 (JPY),0.2696,0.2806,0.2760,0.2800
102,2017/08/30,日圓 (JPY),0.2657,0.2767,0.2721,0.2761
88,2017/09/19,日圓 (JPY),0.2613,0.2723,0.2677,0.2717


### 將掛牌日期改為Datetime格式，並設為index

In [80]:
from datetime import datetime
jpy_his = jpy_his.set_index(jpy_his['掛牌日期'])
del jpy_his['掛牌日期']
jpy_his.sample(5)

,幣別,現金-本行買入,現金-本行賣出,即期-本行買入,即期-本行賣出
掛牌日期,,,,,
2017/12/19,日圓 (JPY),0.2581,0.2691,0.2645,0.2685
2017/08/22,日圓 (JPY),0.2687,0.2797,0.2751,0.2791
2017/07/25,日圓 (JPY),0.2644,0.2754,0.2708,0.2748
2017/12/29,日圓 (JPY),0.2558,0.2668,0.2622,0.2662
2017/09/27,日圓 (JPY),0.2604,0.2714,0.2668,0.2708


In [81]:
jpy_his.index = pd.to_datetime(jpy_his.index, format='%Y/%m/%d')
jpy_his.sample(5)

,幣別,現金-本行買入,現金-本行賣出,即期-本行買入,即期-本行賣出
掛牌日期,,,,,
2017-09-15,日圓 (JPY),0.2632,0.2742,0.2696,0.2736
2017-11-10,日圓 (JPY),0.2573,0.2683,0.2637,0.2677
2017-12-28,日圓 (JPY),0.2558,0.2668,0.2622,0.2662
2018-01-12,日圓 (JPY),0.2573,0.2683,0.2637,0.2677
2017-11-03,日圓 (JPY),0.2560,0.2670,0.2624,0.2664


In [82]:
type(jpy_his.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [83]:
jpy_his.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 129 entries, 2018-01-24 to 2017-07-25
Data columns (total 5 columns):
幣別         129 non-null object
現金-本行買入    129 non-null float64
現金-本行賣出    129 non-null float64
即期-本行買入    129 non-null float64
即期-本行賣出    129 non-null float64
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


## 將歷即期史匯率繪成圖表

In [84]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.tools.set_credentials_file(username='s842563', api_key='gDEK6deJwLf0LwhD9BzC')

trace_bank_sale = go.Scatter(x = jpy_his.index,
                y=jpy_his['即期-本行賣出'],
                name = "即期匯率-本行賣出",
                line = dict(color = '#17BECF'),
                opacity = 0.8)

trace_bank_buy = go.Scatter(x = jpy_his.index,
                y=jpy_his['即期-本行買入'],
                name = "即期匯率-本行買入",
                line = dict(color = 'red'),
                opacity = 0.8)

data = [trace_bank_sale, trace_bank_buy]

layout = dict(
    title = jpy_his['幣別'][0],
    xaxis = dict(range = [jpy_his.index[-1], jpy_his.index[0]])
             )

fig = dict(data=data, layout=layout)

py.iplot(fig, filename = jpy_his['幣別'][0])

## 將歷史匯率建入資料庫

In [85]:
import sqlite3

### 與資料庫建立連線

In [86]:
with sqlite3.connect('BOT_JPY.sqlite') as db:
    jpy_his.to_sql('BOT_JPY', con= db, if_exists= 'append')

In [14]:
import matplotlib.pyplot as plt
%matplotlib inline

In [76]:
# 作圖風格
plt.style.use('ggplot')
# 作圖大小
fig = plt.figure(figsize=(20,6))
# 字體設定， plt.rcParams['font.family'] = 'DFKai-SB'
font = {'fontname':'DFKai-SB'}

# 資料
y = jpy_his['即期-本行買入']
y2 = jpy_his['即期-本行賣出']
date_range = jpy_his.date_range(jpy_his.index[-1], jpy_his.index[0]])

# 作圖
jpy_his.plot(kind = 'line', y = y, style = 'b.-')
jpy_his.plot(kind = 'line', y = y2, style = 'r.-')

# 標題
jpy_his.plot(title = jpy_his['幣別'][0])

plt.show()

SyntaxError: invalid syntax (<ipython-input-76-dc5ebfddfded>, line 11)